In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-133711
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-133711


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

amlcompute_cluster_name= "cpu-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes= 4 )
aml_compute= ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os


# Specify parameter sampler
ps = ps = RandomParameterSampling(parameter_space={'C': uniform(0.1, 100), 'max_iter':list(range(100,500,100))} )
    

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor= 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training', entry_script='train.py', compute_target=amlcompute_cluster_name)                                                     

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator= est,
                                    hyperparameter_sampling= ps, 
                                    policy= policy,
                                    primary_metric_name= "Accuracy", 
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE, 
                                    max_total_runs= 20,
                                    max_concurrent_runs=4)

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr= exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output= True)
ssert(hdr.get_status() =="completed")

assertion is always true, perhaps remove parentheses?
assertion is always true, perhaps remove parentheses?
assertion is always true, perhaps remove parentheses?


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_092d8108-0663-41e7-8c52-2f24c74eb335
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_092d8108-0663-41e7-8c52-2f24c74eb335?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-133711/workspaces/quick-starts-ws-133711

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-06T21:49:06.606422][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-06T21:49:06.766786][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2021-01-06T21:49:06.184429][API][INFO]Experiment created<END>\n"<START>[2021-01-06T21:49:07.5078273Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_092d8108-0663-41e7-8c52-2f24c74eb335
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_092d8108-0663-41e7-8c52-2f24c74eb335?wsid=/subscriptions/9e65f

assertion is always true, perhaps remove parentheses?


NameError: name 'get_status' is not defined

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run= hdr.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguements'])
print(best_run.get_file_names())
model= best_run.register_model(model_name= 'MLopt_best_model', model_path= 'outputs/model.joblib')

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds= TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv' )

In [26]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

TypeError: 'NoneType' object is not iterable

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= "classification",
    primary_metric= "Accuracy",
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [2]:
# Submit your automl run

remote_run= experiment.submit(automl_config, show_output= False)

In [ ]:
# Retrieve and save your best automl model.

import joblib
# Get your best run and save the model from that run.

best_run= remote_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguements'])

model= best_run.register_model(model_name= 'best_model', model_path= 'outputs/model')